# 3D-CNN Model

In [1]:
import pandas as pd

BH_tif_400_400_after_processing = pd.read_csv('useful_data_after_processing/BH_tif_400_400_after_processing.csv',
                                              sep=',', header=None)
BV_tif_400_400_after_processing = pd.read_csv('useful_data_after_processing/BV_tif_400_400_after_processing.csv',
                                              sep=',', header=None)
CNM_tif_400_400_after_processing = pd.read_csv('useful_data_after_processing/CNM_tif_400_400_after_processing.csv',
                                               sep=',', header=None)
LAI_tif_400_400_after_processing = pd.read_csv('useful_data_after_processing/LAI_tif_400_400_after_processing.csv',
                                               sep=',', header=None)
DSM_tif_400_400_after_processing = pd.read_csv('useful_data_after_processing/DSM_tif_400_400_after_processing.csv',
                                               sep=',', header=None)
Weather_1795_6_after_processing = pd.read_csv('useful_data_after_processing/weather_1795_6.csv', sep=',')


In [2]:
Weather_1795_6_after_processing.shape

(1795, 6)

In [3]:
import numpy as np

In [4]:
X = np.stack([BH_tif_400_400_after_processing, BV_tif_400_400_after_processing, CNM_tif_400_400_after_processing,
              LAI_tif_400_400_after_processing, DSM_tif_400_400_after_processing], axis=-1)

In [5]:
X.shape

(400, 400, 5)

In [6]:
Y = Weather_1795_6_after_processing['tempMax']

In [7]:
Y.shape

(1795,)

In [8]:
X_loaded = X
Y_loaded = Y
X_loaded.shape, Y_loaded.shape

((400, 400, 5), (1795,))

In [9]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

X_original = X_loaded  # X_original 是形状为 [400, 400, 5] 的空间数据
Y_original = Y_loaded  # Y_original 是形状为 [1795,] 的温度时间序列数据
seq_length = 20  # 设定时间序列的长度


In [10]:
X_original.shape, Y_original.shape

((400, 400, 5), (1795,))

In [11]:
# import torch
# from torch.utils.data import Dataset, DataLoader
# 
# class TemporalSpatialDataset(Dataset):
#     def __init__(self, X_spatial, Y_temporal, seq_length):
#         self.X_spatial = X_spatial  # 空间数据，形状 [400, 400, 5]
#         self.Y_temporal = Y_temporal  # 时间序列数据，形状 [1795,]
#         self.seq_length = seq_length
# 
#     def __len__(self):
#         return len(self.Y_temporal) - self.seq_length
# 
#     def __getitem__(self, idx):
#         # 获取当前时间步的输入序列
#         Y_seq = self.Y_temporal[idx:idx + self.seq_length]
#         Y_target = self.Y_temporal[idx + self.seq_length]
# 
#         # 转换为 PyTorch 张量
#         Y_seq = torch.tensor(Y_seq, dtype=torch.float32).unsqueeze(-1)  # 形状 [seq_length, 1]
#         Y_target = torch.tensor(Y_target, dtype=torch.float32).unsqueeze(-1)  # 形状 [1]
# 
#         # 动态生成 X_spatial 输入，重复 seq_length 次
#         X_spatial = torch.tensor(self.X_spatial, dtype=torch.float32).permute(2, 0, 1)  # 形状 [5, 400, 400]
#         X_spatial = X_spatial.unsqueeze(0).repeat(self.seq_length, 1, 1, 1)  # 形状 [seq_length, 5, 400, 400]
# 
#         return X_spatial, Y_seq, Y_target


In [12]:
class TemporalSpatialDataset:
    def __init__(self, X_spatial, Y_temporal, seq_length):
        self.X_spatial = X_spatial
        self.Y_temporal = Y_temporal
        self.seq_length = seq_length

    def __len__(self):
        return len(self.Y_temporal) - self.seq_length

    def __getitem__(self, idx):
        # 获取当前时间步的时间序列部分
        Y_seq = self.Y_temporal[idx: idx + self.seq_length]
        Y_target = self.Y_temporal[idx + self.seq_length]

        # 将时间序列数据转换为 PyTorch 张量
        Y_seq = torch.tensor(np.array(Y_seq), dtype=torch.float32).unsqueeze(-1)  # 形状 [seq_length, 1]
        Y_target = torch.tensor(np.array(Y_target), dtype=torch.float32).unsqueeze(-1)  # 形状 [1]

        # 动态生成 X_spatial 输入，X_spatial 的形状应该是 [5, 400, 400]
        X_spatial = torch.tensor(np.array(self.X_spatial), dtype=torch.float32).permute(2, 0, 1)  # 形状 [5, 400, 400]

        # 重复空间数据 seq_length 次，形成 [5, seq_length, 400, 400]
        X_spatial = X_spatial.unsqueeze(1).repeat(1, self.seq_length, 1, 1)  # 形状 [5, 20, 400, 400]
        X_spatial = X_spatial.permute(0, 2, 3, 1)  # 形状调整为 [5, 400, 400, 20]
        X_spatial = X_spatial.unsqueeze(0)  # 添加 batch 维度，形状 [1, 5, 400, 400, 20]
        X_spatial = X_spatial.permute(0, 1, 4, 2, 3)  # 调整维度为 [1, 5, 20, 400, 400]

        return X_spatial.squeeze(0), Y_seq, Y_target


In [13]:
dataset = TemporalSpatialDataset(X_original, Y_original, seq_length)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

In [14]:
import torch.nn.functional as F

class CNN3DModel(nn.Module):
    def __init__(self, input_channels, seq_length):
        super(CNN3DModel, self).__init__()
        self.conv1 = nn.Conv3d(in_channels=input_channels, out_channels=32, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.conv2 = nn.Conv3d(in_channels=32, out_channels=64, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.conv3 = nn.Conv3d(in_channels=64, out_channels=128, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool = nn.MaxPool3d(kernel_size=(2, 2, 2))
        self.fc1 = nn.Linear(128 * (seq_length // 2 // 2 // 2) * (400 // 2 // 2 // 2) * (400 // 2 // 2 // 2), 128)
        self.fc2 = nn.Linear(128, 1)  # 预测一个标量，作为 Y_target 的预测

    def forward(self, x):
        print(f'Start CNN3DModel training, shape of x: {x.shape}')
        
        print(f'Start training layer 1')
        x = self.pool(F.relu(self.conv1(x)))  # 第一层卷积 + 池化
        
        print(f'Start training layer 2')
        x = self.pool(F.relu(self.conv2(x)))  # 第二层卷积 + 池化
        
        print(f'Start training layer 3')
        x = self.pool(F.relu(self.conv3(x)))  # 第三层卷积 + 池化
        
        print(f'Start flatten')
        x = x.view(x.size(0), -1)  # 展平
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [15]:
# 初始化模型
model = CNN3DModel(input_channels=5, seq_length=seq_length)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()


In [ ]:
# 训练循环
num_epochs = 1
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (X_spatial, Y_seq, Y_target) in enumerate(train_loader):
        print(f'Batch {batch_idx+1}/{len(train_loader)} in epoch {epoch+1}/{num_epochs}')
        
        # 确保 X_spatial 的形状正确
        print(f"X_spatial.shape: {X_spatial.shape}")
        print(f"Y_seq.shape: {Y_seq.shape}")
        print(f"Y_target.shape: {Y_target.shape}")

        optimizer.zero_grad()
        
        print(f'Start training')
        output = model(X_spatial)
        
        print(f'Start calculating loss')
        loss = criterion(output, Y_target)
        
        print(f'Start calculating gradients')
        loss.backward()
        
        print(f'Start updating parameters')
        optimizer.step()

        # if batch_idx % 10 == 0:  # 每10个批次输出一次损失
        #     print(f'Epoch {epoch+1}/{num_epochs}, Batch {batch_idx +1 }/len(train_loader), Loss: {loss.item():.4f}')
        
        print(loss.item())

Batch 1/45 in epoch 1/1
X_spatial.shape: torch.Size([32, 5, 20, 400, 400])
Y_seq.shape: torch.Size([32, 20, 1])
Y_target.shape: torch.Size([32, 1])
Start training
Start CNN3DModel training, shape of x: torch.Size([32, 5, 20, 400, 400])
Start training layer 1
Start training layer 2
Start training layer 3
Start flatten
Start calculating loss
Start calculating gradients


# 3D-CNN Model GPU

In [9]:
X_loaded = X
Y_loaded = Y
X_loaded.shape, Y_loaded.shape

((400, 400, 5), (1795,))

In [10]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

X_original = X_loaded  # X_original 是形状为 [400, 400, 5] 的空间数据
Y_original = Y_loaded  # Y_original 是形状为 [1795,] 的温度时间序列数据
seq_length = 20  # 设定时间序列的长度

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

# 检查GPU是否可用并选择设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 定义数据集类
class TemporalSpatialDataset(Dataset):
    def __init__(self, X_spatial, Y_temporal, seq_length):
        self.X_spatial = X_spatial
        self.Y_temporal = Y_temporal
        self.seq_length = seq_length

    def __len__(self):
        return len(self.Y_temporal) - self.seq_length

    def __getitem__(self, idx):
        Y_seq = self.Y_temporal[idx: idx + self.seq_length]
        Y_target = self.Y_temporal[idx + self.seq_length]

        Y_seq = torch.tensor(np.array(Y_seq), dtype=torch.float32).unsqueeze(-1)  # 形状 [seq_length, 1]
        Y_target = torch.tensor(np.array(Y_target), dtype=torch.float32).unsqueeze(-1)  # 形状 [1]

        X_spatial = torch.tensor(np.array(self.X_spatial), dtype=torch.float32).permute(2, 0, 1)  # 形状 [5, 400, 400]
        X_spatial = X_spatial.unsqueeze(1).repeat(1, self.seq_length, 1, 1)  # 形状 [5, 20, 400, 400]
        X_spatial = X_spatial.permute(0, 2, 3, 1)  # 形状调整为 [5, 400, 400, 20]
        X_spatial = X_spatial.unsqueeze(0)  # 添加 batch 维度，形状 [1, 5, 400, 400, 20]
        X_spatial = X_spatial.permute(0, 1, 4, 2, 3)  # 调整维度为 [1, 5, 20, 400, 400]

        return X_spatial.squeeze(0), Y_seq, Y_target

# 初始化数据集和数据加载器
dataset = TemporalSpatialDataset(X_original, Y_original, seq_length)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)  # 批大小调整为64
val_loader = DataLoader(val_dataset, batch_size=64)

# 定义模型
class CNN3DModel(nn.Module):
    def __init__(self, input_channels, seq_length):
        super(CNN3DModel, self).__init__()
        self.conv1 = nn.Conv3d(in_channels=input_channels, out_channels=32, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.conv2 = nn.Conv3d(in_channels=32, out_channels=64, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.conv3 = nn.Conv3d(in_channels=64, out_channels=128, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool = nn.MaxPool3d(kernel_size=(2, 2, 2))
        self.fc1 = nn.Linear(128 * (seq_length // 2 // 2 // 2) * (400 // 2 // 2 // 2) * (400 // 2 // 2 // 2), 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [13]:

# 将模型加载到指定设备上
model = CNN3DModel(input_channels=5, seq_length=seq_length).to(device)

# 定义优化器和损失函数
optimizer = torch.optim.Adam(model.parameters(), lr=0.002)  # 学习率加倍
criterion = nn.MSELoss()

# 使用自动混合精度训练
scaler = torch.amp.GradScaler() if device.type == 'cuda' else None



In [14]:
# 训练循环
num_epochs = 1
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for batch_idx, (X_spatial, Y_seq, Y_target) in enumerate(train_loader):
        X_spatial, Y_target = X_spatial.to(device), Y_target.to(device)

        optimizer.zero_grad()

        with torch.amp.autocast(device_type=device.type, enabled=(device.type == 'cuda')):  # AMP 训练开启
            output = model(X_spatial)
            loss = criterion(output, Y_target)

        scaler.scale(loss).backward()  # AMP 梯度缩放
        scaler.step(optimizer)
        scaler.update()

        train_loss += loss.item()
        print(f'EPOCH {epoch + 1} / {num_epochs} - Batch {batch_idx + 1} / {len(train_loader)} - Loss: {loss.item()}')

        # if batch_idx % 10 == 0:
        #     print(f'Epoch {epoch+1}/{num_epochs}, Batch {batch_idx+1}/{len(train_loader)}, Loss: {loss.item():.4f}')

    print(f'Training Loss: {train_loss / len(train_loader)}')
        

OutOfMemoryError: CUDA out of memory. Tried to allocate 3.05 GiB. GPU 0 has a total capacity of 8.00 GiB of which 0 bytes is free. Of the allocated memory 32.29 GiB is allocated by PyTorch, and 5.62 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)